In [ ]:
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
import pandas as pd

SCOPES = ['https://www.googleapis.com/auth/gmail.modify']
# Step 1. Authenticate and build Gmail service
def get_gmail_service(token_path="token.json", creds_path="credentials.json"):
    creds = None

    # Load existing token if available
    if os.path.exists(token_path):
        creds = Credentials.from_authorized_user_file(token_path, SCOPES)

    # If no valid token, go through OAuth flow
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(creds_path, SCOPES)
            creds = flow.run_local_server(port=0)  # opens a browser for login
        # Save token for next time
        with open(token_path, 'w') as token:
            token.write(creds.to_json())

    return build("gmail", "v1", credentials=creds)

# Step 2. Fetch emails
def fetch_emails(service, max_results=100):
    results = service.users().messages().list(userId="me", maxResults=max_results).execute()
    return results.get("messages", [])

# Step 3. Classify emails
def classify_email(subject, sender):
    delete_keywords = ["unsubscribe", "promo", "sale", "offer", "newsletter"]
    keep_keywords = ["invoice", "bank", "tax", "contract", "family"]

    subject_lower = subject.lower()
    if any(word in subject_lower for word in delete_keywords):
        return "DELETE"
    if any(word in subject_lower for word in keep_keywords):
        return "KEEP"
    return "REVIEW"

# Step 4. Export to CSV
def export_emails(service, emails, filename="emails_review.csv"):
    email_data = []
    for e in emails:
        msg = service.users().messages().get(userId="me", id=e["id"]).execute()
        headers = {h["name"]: h["value"] for h in msg["payload"]["headers"]}
        subject = headers.get("Subject", "")
        sender = headers.get("From", "")
        decision = classify_email(subject, sender)
        email_data.append({"Subject": subject, "Sender": sender, "Decision": decision})

    df = pd.DataFrame(email_data)
    df.to_csv(filename, index=False)
    print(f"Exported {len(df)} emails to {filename}")

In [ ]:
import os

# service = get_gmail_service()

In [ ]:
# emails = fetch_emails(service, max_results=2000)  # adjust number if you want more
# len(emails)

In [ ]:
# export_emails(service, emails, filename="emails_review.csv")

In [1]:
import pandas as pd

df = pd.read_csv("emails_review.csv")

In [1]:
import pandas as pd

df = pd.read_csv("emails_review.csv")

In [2]:
from src.agent.email_agent import EmailAgent

agent = EmailAgent()

In [5]:
updated_df = await agent.run(df[0:10])

  0%|          | 0/10 [00:00<?, ?it/s]

Based on the provided classification rules and email information, here's my response:

**IMPORTANT:** No
**REASON:** The subject mentions a celebration in October, which is likely related to social justice or advocacy content. Although it doesn't explicitly mention dates, events, or calls to action, its tone suggests promotional intent.
**CONFIDENCE:** Medium

This email seems more like a community announcement than something that requires your direct attention. I would recommend filtering this out as spam/promotional.
Based on the provided classification rules and email information, here's my response:

**IMPORTANT:** No
**REASON:** The subject "EV Charging Stations" is related to a service offered by an apartment complex, but it does not indicate any personal or work-related matter that requires direct action. It seems more like promotional content.
**CONFIDENCE:** High

This email can be filtered out as spam/promotional and removed from your inbox.
Based on the provided classificati

100%|██████████| 10/10 [07:56<00:00, 47.61s/it]  

Based on the provided classification rules and email information, here's my response:

**IMPORTANT:** Yes
**REASON:** The subject mentions an event happening tomorrow, which implies a time-sensitive matter that requires your direct action or attention.
**CONFIDENCE:** High

I'm confident in this assessment because the subject explicitly states "tomorrow," indicating urgency. This is likely to be important for you as it relates to personal matters (assuming the sender has some relevance).
[{'important': 'NO', 'reason': 'The subject line "Has your score changed Tamim?" suggests a promotional inquiry about credit scores from Credit Karma. This aligns with rule 3 (emails about credit reports, credit scores, or credit monitoring are NOT important).', 'confidence': 'HIGH'}, {'important': 'NO', 'reason': 'The subject mentions a protest related to social justice (Palestine), which falls under rule 2. This type of content is not considered important.', 'confidence': 'HIGH'}, {'important': 'No',

In [6]:
updated_df

,Subject,Sender,Decision,important,reason,confidence
0,Has your score changed Tamim?,Credit Karma <notifications@creditkarma.ca>,REVIEW,NO,"The subject line ""Has your score changed Tamim...",HIGH
1,Two Years of Genocide: Protest with us on Octo...,"""Neighbours for Palestine: Waterloo Region"" <i...",REVIEW,NO,The subject mentions a protest related to soci...,HIGH
2,🧡 💜 ❤️ Celebrate in October,Waterloo Public Library <socialteam@wpl.ca>,REVIEW,No,"The subject mentions a celebration in October,...",Medium
3,🔐 Crack the Code. Win Big. Celebrate Escape Ro...,Escapology Waterloo <waterloo@escapology.com>,REVIEW,NO,The email is a promotional message for Escapol...,HIGH
4,👻& 🎃 LINDOR Truffles are baaaaaack,Lindt Chocolate <lindtca@newsletter.lindt.com>,REVIEW,Yes,The subject line suggests a promotional offer ...,Medium
5,Filing Cabinets. Are. Here.,Wayfair Canada <editor@members.wayfair.ca>,REVIEW,Yes,The subject line mentions a specific product (...,Medium
6,Outerwear for all from $45 🍂,Walmart Canada <offers@e.walmart.ca>,REVIEW,No,The subject line mentions a promotional offer ...,High
7,EV Charing Stations,Northfield And Davenport Apartments <noreply@1...,REVIEW,No,"The subject ""EV Charging Stations"" is related ...",High
8,Psst! Walmart Deals start on Oct 2 ⏰,Walmart Canada <offers@e.walmart.ca>,REVIEW,No,The subject line mentions a specific date (Oct...,Medium
9,1 event happening tomorrow,"""The Green Pill School (Skool)"" <noreply@skool...",REVIEW,Yes,The subject mentions an event happening tomorr...,High


In [ ]:
# import pandas as pd

# def delete_emails_from_csv(service, filename="emails_review.csv"):
#     # Load the reviewed CSV
#     df = pd.read_csv(filename)
# 
#     # Ensure we only pick rows marked DELETE
#     to_delete = df[df["Decision"].str.upper() == "DELETE"]
# 
#     print(f"Found {len(to_delete)} emails marked for deletion.")
# 
#     for _, row in to_delete.iterrows():
#         msg_id = row.get("Message_ID")  # This must be included in the export step
#         if pd.isna(msg_id):
#             continue
# 
#         try:
#             # Gmail API call to delete message
#             service.users().messages().delete(userId="me", id=msg_id).execute()
#             print(f"Deleted email: {row['Subject']} from {row['Sender']}")
#         except Exception as e:
#             print(f"Error deleting {row['Subject']}: {e}")



In [ ]:
# def export_emails(service, emails, filename="emails_review.csv"):
#     email_data = []
#     for e in emails:
#         msg = service.users().messages().get(userId="me", id=e["id"]).execute()
#         headers = {h["name"]: h["value"] for h in msg["payload"]["headers"]}
#         subject = headers.get("Subject", "")
#         sender = headers.get("From", "")
#         decision = classify_email(subject, sender)
#         email_data.append({
#             "Message_ID": e["id"],   # Add this line
#             "Subject": subject,
#             "Sender": sender,
#             "Decision": decision
#         })
# 
#     df = pd.DataFrame(email_data)
#     df.to_csv(filename, index=False)
#     print(f"Exported {len(df)} emails to {filename}")
